# ドロップアウト (Dropout)

In [ ]:
# PyTorchが使うCPUの数を制限します。(VMを使う場合)
%env OMP_NUM_THREADS=1
%env MKL_NUM_THREADS=1

from torch import set_num_threads, set_num_interop_threads
num_threads = 1
set_num_threads(num_threads)
set_num_interop_threads(num_threads)

#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## 過学習(Overtraining)の例
### データ点の生成
sin関数に0.2の大きさのノイズが乗ったデータを考えます。データ数は30にします。
早期終了の例と同様に、scikit learningの関数を使ってデータをトレーニング用と検証用に分割します。

In [ ]:
np.random.seed(0)

nSample = 30

x = np.linspace(-np.pi, np.pi, nSample)
t0 = np.sin(x)
t = t0 + 0.2 * np.random.randn(nSample)

from sklearn.model_selection import train_test_split
x_train, x_valid, t_train, t_valid = train_test_split(x, t, test_size=0.2, random_state=42)

# For plot
x_grid = np.linspace(-np.pi, np.pi, 100)

# numpy -> torch.tensor
from torch import from_numpy
x_train_tensor = from_numpy(x_train).float().unsqueeze(-1)
t_train_tensor = from_numpy(t_train).float().unsqueeze(-1)
x_valid_tensor = from_numpy(x_valid).float().unsqueeze(-1)
t_valid_tensor = from_numpy(t_valid).float().unsqueeze(-1)
x_grid_tensor = from_numpy(x_grid).float().unsqueeze(-1)

### デモに用いる深層学習モデル
過学習の様子を見るために、パラメータ数の多いモデルを使ってフィットをしてみます。
ここでは、隠れ層が5層、ノード数が128の隠れ層を4層重ねた多層パーセプトロンを使用します。
活性化関数としてはReLUを使い、モデルの出力の直前のノードは、活性化関数を使用しないことにします。
誤差関数は二乗和誤差を使い、最適化関数としてadamを使用します。

In [ ]:
from torch.nn import Sequential
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import MSELoss
from torch.optim import Adam

# モデルの定義
model = Sequential(
    Linear(in_features=1, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Linear(in_features=128, out_features=1),  # ノード数が1の層を追加。
)
# 誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
loss_fn = MSELoss()
optimizer = Adam(model.parameters())

loss_train_history = []
loss_valid_history = []

# トレーニング
num_epochs = 300
for i_epoch in range(num_epochs):
    # 順伝搬
    y_pred = model(x_train_tensor)

    # ロスの計算
    loss = loss_fn(y_pred, t_train_tensor)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

    # validationデータセットでロスの値を計算
    y_pred = model(x_valid_tensor)
    loss_valid = loss_fn(y_pred, t_valid_tensor)

    loss_train_history += [loss.detach().numpy()]
    loss_valid_history += [loss_valid.detach().numpy()]

# プロット
plt.scatter(x_train, t_train, s=10, c='black', label='data')  # データ点のプロット
plt.plot(x_grid, model(x_grid_tensor).detach().numpy(), c='red', label='prediction')
plt.plot(x_grid, np.sin(x_grid), c='blue', label='y=sin(r)')
plt.legend()
plt.show()

# ロス関数の推移をプロット
plt.plot(loss_train_history, label='loss (train)')
plt.plot(loss_valid_history, label='loss (valid)')
plt.legend()
plt.show()


## ドロップアウト (Dropout)

ドロップアウトは、学習時に、ランダムにノードを消すという操作をすることで、表現能力を抑制する手法です。
この操作は、アンサンブル学習を擬似的に行っていると見ることもできます。

この操作はPyTorchの関数を用いることで簡単に実装することができます。

https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html

```python
Dropout(p=0.3)
```
ここで、消すノードの割合を`p`で指定します。0だと、ノードは全く消されません。1.0だと、全てのノードが消されます。

ここでは、MLPの各層の後にドロップアウト層を入れることにします。

In [ ]:
from torch.nn import Sequential
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import MSELoss
from torch.nn import Dropout
from torch.optim import Adam

# モデルの定義
model = Sequential(
    Linear(in_features=1, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Dropout(p=0.3),
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Dropout(p=0.3),
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Dropout(p=0.3),
    Linear(in_features=128, out_features=128),  # ノード数が128の層を追加。
    ReLU(),  # 活性化関数はReLU。
    Dropout(p=0.3),
    Linear(in_features=128, out_features=1),  # ノード数が1の層を追加。
)
# 誤差関数としてクロスエントロピーを指定。最適化手法は(確率的)勾配降下法
loss_fn = MSELoss()
optimizer = Adam(model.parameters())

loss_train_history = []
loss_valid_history = []

# トレーニング
num_epochs = 300
for i_epoch in range(num_epochs):
    # trainモードにすることで、dropoutがオンになります。
    model.train()

    # 順伝搬
    y_pred = model(x_train_tensor)

    # ロスの計算
    loss = loss_fn(y_pred, t_train_tensor)

    # 誤差逆伝播の前に各パラメータの勾配の値を0にセットする。
    # これをしないと、勾配の値はそれまでの値との和がとられる。
    optimizer.zero_grad()

    # 誤差逆伝播。各パラメータの勾配が計算される。
    loss.backward()

    # 各パラメータの勾配の値を基に、optimizerにより値が更新される。
    optimizer.step()

    # evalモードにすることで、dropoutがオフになります。
    model.eval()

    # validationデータセットでロスの値を計算
    y_pred = model(x_valid_tensor)
    loss_valid = loss_fn(y_pred, t_valid_tensor)

    loss_train_history += [loss.detach().numpy()]
    loss_valid_history += [loss_valid.detach().numpy()]

# evalモードにすることで、dropoutをオフにします。
model.eval()

# プロット
plt.scatter(x_train, t_train, s=10, c='black', label='data')  # データ点のプロット
plt.plot(x_grid, model(x_grid_tensor).detach().numpy(), c='red', label='prediction')
plt.plot(x_grid, np.sin(x_grid), c='blue', label='y=sin(r)')
plt.legend()
plt.show()

# ロス関数の推移をプロット
plt.plot(loss_train_history, label='loss (train)')
plt.plot(loss_valid_history, label='loss (valid)')
plt.legend()
plt.show()


トレーニングデータに対する誤差関数が大きくふらついていることがわかります。
これは、消されるノードが毎回ランダムに選ばれるためです。
一方で、検証用データではばらつきは比較的小さくなっています。
これは、`model.eval()`を実行したことで、検証用データを使ってモデルの予測をする際は全てのノードを使うようになっているためです。

ドロップアウトの`p`の値を変えたり、ドロップアウトの位置を変えると、モデルの予測はどのように変化するでしょうか？